# DeepFM

이번 실습에서는 DeepFM 모델을 이해하고 구현해보겠습니다.  

DeepFM 모델은 Factorization machines와 neural network를 합친 모델로, Wide & Deep model과 유사하지만, feature engineering이 필요하지 않다는 특징을 가지고 있습니다.  
<br/>
사용자가 영화에 대해 Rating한 데이터, 영화의 장르 데이터를 이용하여 Train/Test data를 생성한 다음, Train data로 학습한 모델을 Test data에 대해 평가해봅니다.   
사용한 데이터는 Implicit feedback data로, 사용자가 시청한 영화(Positive instances)는 rating = 1로 기록됩니다. 따라서 시청하지 않은 영화에 대해 각 유저별로 Negative instances sampling을 진행합니다.   
<br/>
**구현에 앞서, DeepFM 논문을 꼭 읽어보시길 권장합니다.**

* 참고  
    - DeepFM: A Factorization-Machine based Neural Network for CTR Prediction (https://arxiv.org/pdf/1703.04247.pdf)  
    - Wide & Deep Learning for Recommender Systems (https://arxiv.org/pdf/1606.07792.pdf)
    - Factorization Machines (https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5694074)
    - https://d2l.ai/chapter_recommender-systems/deepfm.html

# Modules

In [1]:
import csv
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data preprocessing
0. Dataset 다운로드  
<br/>
1. Rating df 생성  
rating 데이터(train_ratings.csv)를 불러와 [user, item, rating]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
2. Genre df 생성   
genre 정보가 담긴 데이터(genres.tsv)를 불러와 genre이름을 id로 변경하고, [item, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.    
<br/>
3. Negative instances 생성   
rating 데이터는 implicit feedback data(rating :0/1)로, positive instances로 구성되어 있습니다. 따라서 rating이 없는 item중 negative instances를 뽑아서 데이터에 추가하게 됩니다.   
<br/>
4. Join dfs   
rating df와 genre df를 join하여 [user, item, rating, genre]의 컬럼으로 구성된 데이터 프레임을 생성합니다.   
<br/>
5. zero-based index로 mapping   
Embedding을 위해서 user,item,genre를 zero-based index로 mapping합니다.
    - user : 0-31359
    - item : 0-6806
    - genre : 0-17  
<br/>
6. feature matrix X, label tensor y 생성   
[user, item, genre] 3개의 field로 구성된 feature matrix를 생성합니다.   
<br/>
7. data loader 생성

## 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [2]:
# 1. Rating df 생성
rating_data = "/opt/ml/input/data/train/train_ratings.csv"

raw_rating_df = pd.read_csv(rating_data)
raw_rating_df

# implict feedback을 만들기 위해서 rating column 추가
raw_rating_df['rating'] = 1.0 
# time은 사용하지 않기에 드랍
raw_rating_df.drop(['time'],axis=1,inplace=True)

# 각각 user와 item의 set 만들기
users = set(raw_rating_df.loc[:, 'user'])
items = set(raw_rating_df.loc[:, 'item'])

#2. Genre df 생성
genre_data = "/opt/ml/input/data/train/genres.tsv"

raw_genre_df = pd.read_csv(genre_data, sep='\t')
# 아이템별로 하나의 장르만 남게 드랍 ~ 최초로 나오는 것만
raw_genre_df = raw_genre_df.drop_duplicates(subset=['item'])

# 인덱싱
genre_dict = {genre:i for i, genre in enumerate(set(raw_genre_df['genre']))}
raw_genre_df['genre']  = raw_genre_df['genre'].map(lambda x : genre_dict[x]) 

In [3]:
# 3. Negative instance 생성 ~ 현재 implict feedback은 positive밖에 없기에

num_negative = 50   # negative sampling할 수
user_group_dfs = list(raw_rating_df.groupby('user')['item'])    # 유저-아이템 리스트
first_row = True
user_neg_dfs = pd.DataFrame()   # negative sampling 담을 데이터프레임

for u, u_items in tqdm(user_group_dfs):
    # num_negative만큼, user-item에서 없는 아이템을 랜덤하게 추출
    u_items = set(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
    # 그렇게 만들어진 negative sampling데이터를 합치기 ~ 처음 합쳐지는 경우엔 concat이 불가하니, 대체해서 사용
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

# 최종적으로 postive + negative 합치기 ~
raw_rating_df = pd.concat([raw_rating_df, user_neg_dfs], axis = 0, sort=False)

# 4. genre와 rating merge
joined_rating_df = pd.merge(raw_rating_df, raw_genre_df, left_on='item', right_on='item', how='inner')

# 5. user, item을 인덱싱, 이때 각각 unique한 feature set도 생성
users = list(set(joined_rating_df.loc[:,'user']))
users.sort()
items =  list(set((joined_rating_df.loc[:, 'item'])))
items.sort()
genres =  list(set((joined_rating_df.loc[:, 'genre'])))
genres.sort()

# 즐거운 인덱싱
if len(users)-1 != max(users):
    users_dict = {users[i]: i for i in range(len(users))}
    joined_rating_df['user']  = joined_rating_df['user'].map(lambda x : users_dict[x])
    users = list(set(joined_rating_df.loc[:,'user']))
    
if len(items)-1 != max(items):
    items_dict = {items[i]: i for i in range(len(items))}
    joined_rating_df['item']  = joined_rating_df['item'].map(lambda x : items_dict[x])
    items =  list(set((joined_rating_df.loc[:, 'item'])))

joined_rating_df = joined_rating_df.sort_values(by=['user'])
joined_rating_df.reset_index(drop=True, inplace=True)
# 변수 재지정
data = joined_rating_df

# num 지정
n_data = len(data)
n_user = len(users)
n_item = len(items)
n_genre = len(genres)


100%|██████████| 31360/31360 [05:30<00:00, 94.90it/s] 


In [4]:
#6. 각 feature들에 대한 tensor 생성
user_col = torch.tensor(data.loc[:,'user'])
item_col = torch.tensor(data.loc[:,'item'])
genre_col = torch.tensor(data.loc[:,'genre'])

# 각 텐서의 index가 고유한 값을 갖게 최대값을 순차적으로 더해주기 ~ indexing해줄때 해줘도 그만이다
offsets = [0, n_user, n_user+n_item]
for col, offset in zip([user_col, item_col, genre_col], offsets):
    col += offset

# X, y 데이터 생성
X = torch.cat([user_col.unsqueeze(1), item_col.unsqueeze(1), genre_col.unsqueeze(1)], dim=1)
y = torch.tensor(list(data.loc[:,'rating']))


#7. data loader 생성
class RatingDataset(Dataset):
    def __init__(self, input_tensor, target_tensor):
        self.input_tensor = input_tensor.long()
        self.target_tensor = target_tensor.long()

    def __getitem__(self, index):
        return self.input_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.target_tensor.size(0)

# 데이터 셋 만들기 이때, train ratio는 0.9
dataset = RatingDataset(X, y)
train_ratio = 0.9

# 랜덤으로 split
train_size = int(train_ratio * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# loader에 태우기, 배치사이즈 조절 가능, train은 shuffle된 상태
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

   # Model architecture (DeepFM)
   DeepFM 모델은 1) FM component와  2) Deep component가 병렬적으로 결합되어 있습니다. 구조는 다음과 같습니다.
<img src='https://drive.google.com/uc?id=1vwcxUJQTIsg5QH9CuH5PcUEfExhToUHR'>  
각 구조는 다음과 같습니다.  
   **1. FM component**  
       FM component는 우리가 아는 2-way Factorization machines(degree=2)입니다. FM은 variables 간의 interaction을 다음과 같이 모델링 합니다.   
     **<center> equation (1) </center>**
   $$\hat{y}(x):=w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j$$   
   이때, 세번째 interaction term을 전개하여 다음과 같이 쓸 수 있습니다.(논문 참고)  
   구현 코드는 전개된 식을 바탕으로 합니다.   
     **<center> equation (2)> </center>**
   $$\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf{v}_i,\mathbf{v}_j>x_ix_j = \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2-\sum_{i=1}^{n}v_{i,f}^2x_i^2)$$   
           
   **2. Deep component**  
       Deep component는 MLP Layers로 구성되어 있습니다.   
       구현 코드는 Input dimension이 30-20-10인 3 layer MLP 구조입니다.
  
   

In [5]:
class DeepFM(nn.Module):
    def __init__(self, input_dims, embedding_dim, mlp_dims, drop_rate=0.1):
        super(DeepFM, self).__init__()
        
        # MLP레이어 쌓기
        # input_dims = n_user + n_movie + n_genre ~ 모든 feature의 길이 합
        total_input_dim = int(sum(input_dims)) 

        # 초기 bias 설정 및 전체 최종 layer emb 설정
        self.bias = nn.Parameter(torch.zeros((1,)))
        self.fc = nn.Embedding(total_input_dim, 1)
        
        # 들어오는 n개의 피쳐들에 대해서 emb
        self.embedding = nn.Embedding(total_input_dim, embedding_dim) 
        self.embedding_dim = len(input_dims) * embedding_dim

        # MLP dim에 따라 layer 진행
        mlp_layers = []
        for i, dim in enumerate(mlp_dims):
            # mlp레이어 쌓기
            if i==0:
                # 첫 레이어에 대해서는 input으로 들어올 dim과 실제 mlp레이어 dim의 Linear
                mlp_layers.append(nn.Linear(self.embedding_dim, dim))
            else:
                # 그 이후 레이어에 대해서는 이전 레이어의 output과 실제 mlp레이어 dim의 Linear
                 mlp_layers.append(nn.Linear(mlp_dims[i-1], dim))    
            # activation layer 추가
            mlp_layers.append(nn.ReLU(True))                     
            # dropout
            mlp_layers.append(nn.Dropout(drop_rate))
            
        # 최종적으로 마지막에 마지막 레이어의 dim에서 1로 가는 Linear
        mlp_layers.append(nn.Linear(mlp_dims[-1], 1)) 
        #  Sequential로 최종 정리
        self.mlp_layers = nn.Sequential(*mlp_layers)

    def fm(self, x):
        
        # FM 레이어 쌓기
        # embding_dim으로 임베딩 된 값에 대해
        embed_x = self.embedding(x) 

        # bias 더 해주고
        fm_y = self.bias + torch.sum(self.fc(x), dim=1)
        # MSE loss
        # mse를 위해 square sum과 sum square로 설정 
        square_of_sum = torch.sum(embed_x, dim=1) ** 2        
        sum_of_square = torch.sum(embed_x ** 2, dim=1)
        fm_y += 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1, keepdim=True)
        return fm_y
    
    def mlp(self, x):
        
        # MLP레이어 결과본
        # embding_dim으로 임베딩 된 값에 대해
        embed_x = self.embedding(x)
        
        # 사이즈 맞춰서 squeeze해주고 mlp_layer에 태우기
        inputs = embed_x.view(-1, self.embedding_dim)
        mlp_y = self.mlp_layers(inputs)
        return mlp_y

    def forward(self, x):
        
        # 최종 forward
        # embding_dim으로 임베딩 된 값에 대해
        embed_x = self.embedding(x)
        
        # fm 레이어 결과값
        fm_y = self.fm(x).squeeze(1)
        
        # mlp 레이어 결과값
        mlp_y = self.mlp(x).squeeze(1)
        
        # concat후 sigmoid 태우기
        y = torch.sigmoid(fm_y + mlp_y)
        return y


# Training

In [16]:
# 디바이스 설정
device = torch.device('cuda')
# input_dims 설정
input_dims = [n_user, n_item, n_genre]
# 임베딩 dim 설정
embedding_dim = 10
# 모델설정
model = DeepFM(input_dims, embedding_dim, mlp_dims=[30, 20, 10]).to(device)
# loss 설정정
bce_loss = nn.BCELoss() # Binary Cross Entropy loss
# Optimizer 설정 및 epochs 설정
lr, num_epochs = 0.01, 10
optimizer = optim.Adam(model.parameters(), lr=lr)

for e in tqdm(range(num_epochs)) :
    # 각각 user,item,genre / rating
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        output = model(x)
        loss = bce_loss(output, y.float())
        loss.backward()
        optimizer.step()

100%|██████████| 10/10 [14:03<00:00, 84.35s/it]


# Evaluation
평가는 모델이 postive instance에 대해 0.5이상, negative instance에 대해 0.5미만의 값을 예측한 Accuracy를 측정하여 진행됩니다.

In [30]:
# 평가하기 
correct_result_sum = 0
# test loader에 대해서 진행 이때 지표는 acc
for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    model.eval()
    output = model(x)
    result = torch.round(output)
    correct_result_sum += (result == y).sum().float()

acc = correct_result_sum/len(test_dataset)*100
print("Final Acc : {:.2f}%".format(acc.item()))

Final Acc : 90.46%


# inf

In [168]:
u_list = []
i_list = []
ritems_dict = {v:k for k,v in items_dict.items()}

for u, u_items in tqdm(user_group_dfs):

    # 인코딩하기 전에 유저id 저장
    u_list.append([u]*10)

    # user_group_dfs은 인코딩 이전 값이므로 사용하기 위해 인코딩 진행
    u = users_dict[u]
    u_items = set(u_items.map(lambda x : items_dict[x]))    # 나중에 본 것들 제외용

    # user, item, genre 데이터를 인코딩하여 학습한 모델에 맞는 값으로 변환
    i_user_col = torch.tensor([u] * n_item)
    i_item_col = torch.tensor(raw_genre_df['item'].map(lambda x : items_dict[x]).values)
    i_genre_col = torch.tensor(raw_genre_df['genre'].values)
    for col, offset in zip([i_user_col, i_item_col, i_genre_col], offsets):
        col += offset
        
    X = torch.cat([i_user_col.unsqueeze(1), i_item_col.unsqueeze(1), i_genre_col.unsqueeze(1)], dim=1)
    X = X.to(device)
    with torch.no_grad():
        model.eval()
        output = model(X)
        output = output.cpu().detach().numpy()
        output[list(u_items)] = -np.inf   # 이미 본 아이템 제외
        result_batch = np.argsort(output[list(u_items)])[::-1][:10] # 역방향 -> 정방향으로 수정
        i_list.append(list(map(lambda x : ritems_dict[x], result_batch)))  # 아이템 디코딩, ndarray는 map()이 안돼서 다른 방법 찾음
    


  7%|▋         | 2313/31360 [00:11<02:30, 193.22it/s]


KeyboardInterrupt: 

In [149]:
u_list = np.concatenate(u_list)
i_list = np.concatenate(i_list)

submit_df = pd.DataFrame(data={'user': u_list, 'item': i_list}, columns=['user', 'item'])
print()


In [124]:
submit_df.to_csv('/opt/ml/input/sample_code/sub_fm_new.csv')

In [164]:
np.argsort(output[list(u_items)])[::-1][:10]

array([349,   6, 283,  77,   0,  47,   9,  22,  10, 116])

In [166]:
np.argsort(output)[::-1][:10]

array([   1, 2993,   22,   27,    6,  127, 3655, 3419, 3780, 2938])

In [167]:
np.argsort(output)[-10:][::-1]

array([   1, 2993,   22,   27,    6,  127, 3655, 3419, 3780, 2938])

In [135]:
float(9.9998283e-01)

0.99998283